### Подключение библиотек

In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.neighbors import KDTree
import random
import matplotlib.pyplot as plt
import time
import pickle
from IPython.display import Image

### Считаем данные

Теоретические данные

In [29]:
X_test_array = np.load(r"C:\Users\trrak\OneDrive\Рабочий стол\InvBloodSolver\df\df_th\X_test_array.npy")
X_train_array = np.load(r"C:\Users\trrak\OneDrive\Рабочий стол\InvBloodSolver\df\df_th\X_train_array.npy")
y_test_array = np.load(r"C:\Users\trrak\OneDrive\Рабочий стол\InvBloodSolver\df\df_th\y_test_array.npy")
y_train_array = np.load(r"C:\Users\trrak\OneDrive\Рабочий стол\InvBloodSolver\df\df_th\y_train_array.npy")

Экспериментальные данные

In [30]:
X_exp_array = np.load(r"C:\Users\trrak\OneDrive\Рабочий стол\InvBloodSolver\df\df_exp1\X_exp_array.npy")
X_th_array = np.load(r"C:\Users\trrak\OneDrive\Рабочий стол\InvBloodSolver\df\df_exp1\X_th_array.npy")
y_th_array = np.load(r"C:\Users\trrak\OneDrive\Рабочий стол\InvBloodSolver\df\df_exp1\y_th_array.npy")

### Нахождение RMSE (минимального расстояния)

Расчет теоретических данных

In [35]:
# Создание пустого массива нужной длины для сохранения результатов
rmse_results = np.zeros(X_test_array.shape[0])
index_results = np.zeros(X_test_array.shape[0])

# Проход по каждой строке в X_test_array
for i in tqdm(range(X_test_array.shape[0])):
    # Считывание всех RMSE для одной строки X_test_array по X_train_array 
    # Нахождение минимального RMSE и соответствующего индекса для каждой строки X_test_array 
    rmse = np.sqrt(np.mean(np.square(X_train_array - X_test_array[i, :]), axis=1))
    min_rmse = np.min(rmse)
    min_index = np.argmin(rmse)
    # Создание два массива под RMSE и индекс
    rmse_results[i] = min_rmse
    index_results[i] = min_index

100%|██████████| 39229/39229 [35:41<00:00, 18.31it/s]


In [38]:
np.savetxt(r'C:\Users\trrak\OneDrive\Рабочий стол\InvBloodSolver\bd_search_files\index_th.txt', index_results)

In [39]:
index_results

array([ 84362.,  38684.,  53880., ..., 125388., 129208., 132379.])

Расчет экспериментальных данных

In [40]:
# Создание пустого массива нужной длины для сохранения результатов
rmse_results = np.zeros(X_exp_array.shape[0])
index_results = np.zeros(X_exp_array.shape[0])

# Проход по каждой строке в X_test_array
for i in tqdm(range(X_exp_array.shape[0])):
    # Считывание всех RMSE для одной строки X_test_array по X_train_array 
    # Нахождение минимального RMSE и соответствующего индекса для каждой строки X_test_array 
    rmse = np.sqrt(np.mean(np.square(X_th_array - X_exp_array[i, :]), axis=1))
    min_rmse = np.min(rmse)
    min_index = np.argmin(rmse)
    # Создание два массива под RMSE и индекс
    rmse_results[i] = min_rmse
    index_results[i] = min_index

100%|██████████| 17441/17441 [23:24<00:00, 12.42it/s]


In [41]:
np.savetxt(r'C:\Users\trrak\OneDrive\Рабочий стол\InvBloodSolver\bd_search_files\index_exp.txt', index_results)

In [44]:
index_results

array([ 18205., 120062.,  78662., ...,  97390., 190780., 123921.])

Чтение файлов, чтобы повторно не прогонять RMSE

In [45]:
index_th = np.loadtxt(r'C:\Users\trrak\OneDrive\Рабочий стол\InvBloodSolver\bd_search_files\index_th.txt')
index_exp = np.loadtxt(r'C:\Users\trrak\OneDrive\Рабочий стол\InvBloodSolver\bd_search_files\index_exp.txt')

In [46]:
print(index_th.shape)
print(index_exp.shape)

(39229,)
(17441,)


Посмотрим метрику для теоретических данных по выходным параметрам

In [47]:
# Создание пустого массива для сохранения предсказанных значений
y_test_pred_array = np.zeros(y_test_array.shape)

# Проход по каждому индексу в index_results
for i, index in enumerate(index_th):
    # Извлечение соответствующего элемента из y_train_scaled и сохраняем его в y_test_pred_scaled
    y_test_pred_array[i] = y_train_array[int(index)]

# Вычисление MAE для каждой колонки
# multioutput='raw_values' возвращает массив значений MAE для каждой колонки входных данных
mae = mean_absolute_error(y_test_array, y_test_pred_array, multioutput='raw_values')
mse = mean_squared_error(y_test_array, y_test_pred_array, multioutput='raw_values')

# Вывод MAE для каждой колонки
for i, column in enumerate(['r', 'eps', 'n', 'beta']):
    print(f"MAE for {column}: {mae[i]}")
    
# Вывод MSE для каждой колонки
for i, column in enumerate(['r', 'eps', 'n', 'beta']):
    print(f"MSE for {column}: {mse[i]}")

MAE for r: 0.04411429983124591
MAE for eps: 0.029147314678677202
MAE for n: 0.003633561491753548
MAE for beta: 2.023859899564098
MSE for r: 0.004675887140188402
MSE for eps: 0.001749899307930474
MSE for n: 3.0546679281205864e-05
MSE for beta: 15.201713018430244
